In [1]:
using Revise
includet("utils/file_reader.jl")
includet("utils/some_data.jl")
includet("utils/utils_graph.jl")
includet("resolution/compact/compact_formulation.jl")
includet("resolution/twostage/improved_twostage.jl")
using Revise, JuMP, CPLEX



In [2]:
instance = get_instance_from_folder("instances/realistic/2806/easyone/")
#instance = get_instance_from_folder("instances/othersstuff/0207/")



InstanceVNE(Any[Meta graph based on a SimpleDiGraph{Int64} with vertex labels of type Int64, vertex metadata of type Dict, edge metadata of type Dict, graph metadata given by Dict(:type => "virtual", :name => "wheel_3"), and default weight 1.0], Meta graph based on a SimpleDiGraph{Int64} with vertex labels of type Int64, vertex metadata of type Dict, edge metadata of type Dict, graph metadata given by Dict(:type => "substrate", :name => "zoo_Xeex"), and default weight 1.0)

In [21]:
function pricer(instance, pi_value, alpha)
    #### Model
    model_placement = Model(CPLEX.Optimizer)
    set_attribute(model_placement, "CPX_PARAM_EPINT", 1e-8)

    ### Variables
    x_variables = @variable(model_placement, x[v_network in instance.v_networks, vertices(v_network), vertices(instance.s_network)], binary=true);

    ### objective
    placement_cost = @expression(model_placement, sum( 
        instance.s_network[s_node][:cost] * v_network[v_node][:dem] * x[v_network, v_node, s_node] 
        for v_network in instance.v_networks for v_node in vertices(v_network) for s_node in vertices(instance.s_network) )
    )

    additional_cost = @expression(model_placement, sum(
        - alpha[v_network][v_edge][s_node] * (x[v_network, src(v_edge), s_node] - x[v_network, dst(v_edge), s_node] )
        for v_network in instance.v_networks for v_edge in edges(v_network) for s_node in vertices(instance.s_network)
    ))

    @objective(model_placement, Min, -pi_value + placement_cost + additional_cost);
    
    ### Constraints

    # one substrate node per virtual node
    for v_network in instance.v_networks
        for v_node in vertices(v_network)
            @constraint(model_placement, sum(x[v_network, v_node, s_node] for s_node in vertices(instance.s_network)) == 1)
        end
    end

    # node capacity
    for s_node in vertices(instance.s_network)
        @constraint(model_placement, 
            sum( v_network[v_node][:dem] * x[v_network, v_node, s_node] 
                for v_network in instance.v_networks for v_node in vertices(v_network) ) 
            <= 
            instance.s_network[s_node][:cap] )
    end

    # Solving
    set_time_limit_sec(model_placement, 30.)
    set_silent(model_placement)
    optimize!(model_placement)

    # Get the solution
    x_values = value.(x_variables);
    node_placements = Dict()
    for v_network in instance.v_networks
        node_placement = []
        for v_node in vertices(v_network)
            current_placement = []
            for s_node in vertices(instance.s_network)
                if x_values[v_network, v_node, s_node] > 0.99
                    push!(current_placement, 1)
                else
                    push!(current_placement, 0)
                end
            end
            push!(node_placement, current_placement)
        end
        node_placements[v_network] = node_placement
    end
    return node_placements, objective_value(model_placement)
end

pricer (generic function with 1 method)

In [20]:
function add_columns(instance, model_routing, node_placements)
    placement_cost = sum( v_network[v_node][:dem] * instance.s_network[s_node][:cost] * node_placements[v_network][v_node][s_node] for v_network in instance.v_networks for v_node in vertices(v_network) for s_node in vertices(instance.s_network) )

    variable_placement = @variable(model_routing, binary = true)
    set_objective_coefficient(model_routing, variable_placement,  placement_cost)
    set_normalized_coefficient(model_routing[:placement_selection], variable_placement, 1)

    for v_network in instance.v_networks
        placement = node_placements[v_network]
        #println("Placement : " * string(placement))
        #println("cout: " * string(placement_cost))
        for v_edge in edges(v_network)
            for s_node in vertices(instance.s_network)
                #println("Arrete " * string(v_edge) * " et Noeud " * string(s_node) * ", on aura : " * string(placement[src(v_edge)][s_node] - placement[dst(v_edge)][s_node]))
                set_normalized_coefficient(model_routing[:flow_conservation][v_network, v_edge, s_node], variable_placement, placement[src(v_edge)][s_node] - placement[dst(v_edge)][s_node])
            end
        end
    end
end

add_columns (generic function with 1 method)

In [26]:

model_routing = Model(CPLEX.Optimizer)
set_attribute(model_routing, "CPX_PARAM_EPINT", 1e-8)

### Variables
y_variables = @variable(model_routing, y[v_network in instance.v_networks, edges(v_network), edges(instance.s_network)], binary=true);

### Objective
routing_cost = @expression(model_routing, sum( instance.s_network[src(s_edge), dst(s_edge)][:cost] * v_network[src(v_edge), dst(v_edge)][:dem] * y[v_network, v_edge, s_edge] 
    for v_network in instance.v_networks for v_edge in edges(v_network) for s_edge in edges(instance.s_network) ))
@objective(model_routing, Min, routing_cost);


### Constraints

# convexity constraints = placement selection
@constraint(model_routing,
        placement_selection,
        0 
        ==
        1
)

# edge capacity
for s_edge in edges(instance.s_network)
    @constraint(model_routing, 
        sum( v_network[src(v_edge), dst(v_edge)][:dem] * y[v_network, v_edge, s_edge] 
            for v_network in instance.v_networks for v_edge in edges(v_network)) 
        <= 
        instance.s_network[src(s_edge), dst(s_edge)][:cap] )
end

# Flow conservation
@constraint(model_routing,
    flow_conservation[v_network in instance.v_networks, v_edge in edges(v_network), s_node in vertices(instance.s_network)],
    0 
    ==
    sum(y[v_network, v_edge, s_edge] for s_edge in get_out_edges(instance.s_network, s_node)) - 
        sum(y[v_network, v_edge, s_edge] for s_edge in get_in_edges(instance.s_network, s_node))
)

# get original columns
placements_base = get_decent_placements(instance)
add_columns(instance, model_routing, placements_base)

#print(model_routing)

set_time_limit_sec(model_routing, 30.)

keep_on_trying = true
alpha = nothing
pi_value = nothing
while keep_on_trying
    relax_integrality(model_routing)
    set_silent(model_routing)
    optimize!(model_routing)
    println("Objective value: " * string(objective_value(model_routing)))

    pi_value = dual(model_routing[:placement_selection])
    
    alpha = Dict()
    for v_network in instance.v_networks
        alpha[v_network] = Dict()
        for v_edge in edges(v_network)
            alpha[v_network][v_edge] = []
            for s_node in vertices(instance.s_network)
                append!(alpha[v_network][v_edge], dual(model_routing[:flow_conservation][v_network, v_edge, s_node]))
            end
        end
    end

    keep_on_trying = false
    new_placements, value = pricer(instance, pi_value, alpha)
    println("Subproblem value : " * string(value))
    if value < -0.000000000001
        keep_on_trying = true
        add_columns(instance, model_routing, new_placements )
    end
end


Objective value: 34.0
Subproblem value : -46.0
Objective value: 16.0
Subproblem value : -27.0
Objective value: 12.000000000000004
Subproblem value : -28.0
Objective value: 10.666666666666668
Subproblem value : -30.666666666666664
Objective value: 9.999999999999996
Subproblem value : -31.0
Objective value: 7.666666666666671
Subproblem value : -28.555555555555557
Objective value: 7.5555555555555545
Subproblem value : -33.666666666666664
Objective value: 6.055555555555557
Subproblem value : -21.999999999999996
Objective value: 6.055555555555558
Subproblem value : -19.194444444444443
Objective value: 6.055555555555557
Subproblem value : -23.444444444444443
Objective value: 6.055555555555557
Subproblem value : -18.805555555555557
Objective value: 5.846153846153846
Subproblem value : -21.115384615384617
Objective value: 5.82716049382716
Subproblem value : -22.67813051146385
Objective value: 5.675675675675672
Subproblem value : -22.027027027027025
Objective value: 5.40625
Subproblem value : -

In [6]:
pi_value

34.0